In [ ]:

import tensorflow as tf
tf.compat.v1.enable_eager_execution()
import numpy as np
# from tensorflow.keras import layers, models, Sequential, Conv2D, MaxPooling2D, Dropout
from keras.models import Sequential, Model
from keras.layers import LSTM, Dense, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, concatenate, Input
import matplotlib.pyplot as plt

In [ ]:
import os

import cv2
from keras import backend as K
from keras.preprocessing import image
from keras.applications import imagenet_utils
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_resnet50
from tensorflow.keras.applications.vgg19 import preprocess_input as preprocess_vgg19
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
# from tensorflow._api.v2.compat.v1 import float32
# # Load DREAMS data and devide into 2 class
# import cv2
# from google.colab.patches import cv2_imshow
# import os
# excerpt_data_path = r'/content/drive/MyDrive/DREAMS_data/SMOTE_5s_based_regenerated_balanced_data/compare_1D&2D/SMOTE_5s_epoch_based_with_without_REMs_phase_shift_80%_overlap_plot'
# excerpts_data_folder_name = os.listdir(excerpt_data_path)

# # load data
# img_size = (128, 128)
# RGB_img_size = (128, 128, 3)
# with_REMs_img = np.zeros(RGB_img_size, dtype='float32')
# without_REMs_img = np.zeros(RGB_img_size, dtype='float32')
# with_REMs_img = np.expand_dims(with_REMs_img, axis=0)
# without_REMs_img = np.expand_dims(without_REMs_img, axis=0)


# for folder_name in excerpts_data_folder_name:
#   img_names = os.listdir(excerpt_data_path + '/' + folder_name)
#   # print(img_names)
#   for name in img_names:
#     if name[-5] == '1':
#       # print(name)
#       img = cv2.imread(excerpt_data_path + '/' + folder_name + '/' + name)
#       img = cv2.resize(img, img_size, interpolation=cv2.INTER_AREA)
#       img = np.expand_dims(img, axis=0)
#       with_REMs_img = np.append(with_REMs_img, img, axis=0)
#       print(with_REMs_img.shape)
#     elif name[-5] == '0':
#       # print(name)
#       img = cv2.imread(excerpt_data_path + '/' + folder_name + '/' + name)
#       img = cv2.resize(img, img_size, interpolation=cv2.INTER_AREA)
#       img = np.expand_dims(img, axis=0)
#       without_REMs_img = np.append(without_REMs_img, img, axis=0)
#       print(without_REMs_img.shape)

# with_REMs_img = preprocess_resnet50(with_REMs_img)
# without_REMs_img = preprocess_resnet50(without_REMs_img)
# # print('Summary')
# # print(with_REMs_img.shape)
# # print(without_REMs_img.shape)
# # with_REMs_img = np.delete(with_REMs_img, 0, axis=0)
# # without_REMs_img = np.delete(without_REMs_img, 0, axis=0)
# # print(with_REMs_img.shape)
# # print(without_REMs_img.shape)



In [ ]:
print('with REMs')
print(with_REMs_img)
print('without REMs')
print(without_REMs_img)

In [ ]:
print('with REMs')
for i in range(0,60):
  cv2_imshow(with_REMs_img[20+i,:,:,:])
  print('')

print('without REMs')
for i in range(0,60):
  cv2_imshow(without_REMs_img[20+i,:,:,:])
  print('')

In [ ]:
# Shuffle divided-into-two-class data
from sklearn.utils import shuffle
with_REMs_img = shuffle(with_REMs_img, random_state=0)
without_REMs_img = shuffle(without_REMs_img, random_state=0)
with_REMs_img_amount = with_REMs_img.shape[0]
selected_img = np.append(with_REMs_img, without_REMs_img[:with_REMs_img_amount,:,:,:], axis=0)
selected_img_label = np.append(np.ones((with_REMs_img_amount,), dtype='int'), np.zeros((with_REMs_img_amount,), dtype='int'))
print(selected_img.shape)
print(selected_img)
print(selected_img_label[1:30])
selected_img, selected_img_label = shuffle(selected_img, selected_img_label, random_state=0)
print(selected_img_label[1:30])

In [ ]:
for i in range(0,60):
  cv2_imshow(selected_img[20+i,:,:,:])
  print('')

In [ ]:
# DREAMS data(balance data) partition
train_amount = int(selected_img.shape[0] * 80/100)
valid_amount = int(selected_img.shape[0] * 10/100)
test_amount = int(selected_img.shape[0] * 10/100)
train_images = selected_img[0:train_amount,:,:,:]
train_labels = selected_img_label[0:train_amount]
valid_images = selected_img[train_amount:train_amount + valid_amount,:,:,:]
valid_labels = selected_img_label[train_amount:train_amount + valid_amount]
test_images = selected_img[train_amount + valid_amount:train_amount + valid_amount + test_amount,:,:,:]
test_labels = selected_img_label[train_amount + valid_amount:train_amount + valid_amount + test_amount]

In [ ]:
for i in range(0,60):
  cv2_imshow(test_images[20+i,:,:,:])
  print('')

# Load Coherence heatmap data

In [ ]:
# Load DREAMS data(with/without REMs)(whole)
import cv2
# from google.colab.patches import cv2_imshow
import os
# from natsort import natsorted
excerpt_data_path = r'/kaggle/input/5s-rems-slidingwindow-noise-augment-coherence-plot/5s_epoch_based_with_without_REMs_sliding_window_whiteNoise_augmented_coherence_plot/5s_epoch_based_with_without_REMs_sliding_window_whiteNoise_augmented_coherence_plot'
excerpts_data_folder_name = os.listdir(excerpt_data_path)
excerpts_data_folder_name = sorted(excerpts_data_folder_name)


# load data
img_size = (128, 128)


#Subjects data start_index & end_index mark
subjects_data_index_mark = np.zeros((len(excerpts_data_folder_name),2), dtype='int')
subject_count = 0
index = 0
img_count = 0

img_names = os.listdir(excerpt_data_path + '/' + excerpts_data_folder_name[0])
# img_names = sorted(img_names)
# print(img_names[0])
subjects_data_index_mark[subject_count, 0] = index
img_count += 1
if img_names[0][-5] == '1':
    labels = 1
elif img_names[0][-5] == '0':
    labels = 0
else:
    print(img_names[0][-5])
labels = np.expand_dims(labels, axis=0)  
print(labels.shape)
images = cv2.imread(excerpt_data_path + '/' + excerpts_data_folder_name[0] + '/' + img_names[0])
# cv2_imshow(images) 
images = cv2.resize(images, img_size, interpolation=cv2.INTER_AREA)
# cv2_imshow(images)
# print(images)
images = np.expand_dims(images, axis=0)
print(images.shape)
print(labels.shape)
# print(preprocess_images.shape)
for name in img_names[1:]:
#     print(name)
    img_count += 1
    if name[-5] == '1':
        label = 1
    elif name[-5] == '0':
        label = 0
    else:
        print(name[-5])
    label = np.expand_dims(label, axis=0)  
    print(labels.shape)
    print(label.shape)
    labels = np.append(labels, label, axis = 0)
    img = cv2.imread(excerpt_data_path + '/' + excerpts_data_folder_name[0] + '/' + name)
    img = cv2.resize(img, img_size, interpolation=cv2.INTER_AREA)
    img = np.expand_dims(img, axis=0)
    images = np.append(images, img, axis=0)
    print(images.shape)
    print(labels.shape)
subjects_data_index_mark[subject_count, 1] = index + img_count
index = index + img_count
subject_count += 1
    

for folder_name in excerpts_data_folder_name[1:]:
    img_names = os.listdir(excerpt_data_path + '/' + folder_name)
#     img_names = sorted(img_names)
    subjects_data_index_mark[subject_count, 0] = index
    img_count = 0
    for name in img_names[0:]:
        print(name)
        img_count += 1
        if name[-5] == '1':
            label = 1
        elif name[-5] == '0':
            label = 0
        else:
            print(name[-5])
        label = np.expand_dims(label, axis=0)
        labels = np.append(labels, label, axis = 0)
        img = cv2.imread(excerpt_data_path + '/' + folder_name + '/' + name)
        img = cv2.resize(img, img_size, interpolation=cv2.INTER_AREA)
        img = np.expand_dims(img, axis=0)
        images = np.append(images, img, axis=0)
        print(images.shape)
        print(labels.shape)
    subjects_data_index_mark[subject_count, 1] = index + img_count
    index = index + img_count
    subject_count += 1

print(subjects_data_index_mark)
# preprocess_images = preprocess_resnet50(images)
# print(preprocess_images)
# print(preprocess_images.shape)


In [ ]:
from sklearn.metrics import classification_report

import itertools
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import numpy as np

# Confusion matrix
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
# 透過convert_to_labels 將one-hot-encode的資料轉成label
def convert_to_labels(X):
  return np.argmax(X, axis=1)

# Leave-one-subject-out cross validation

In [ ]:
#LOSO validation
# from tensorflow.keras.applications.vgg19 import preprocess_input as preprocess_vgg19
# from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_resnet50
from sklearn.utils import shuffle

# Define per-subject-left-out score containers
acc_per_subject_left_out = []
loss_per_subject_left_out = []


# Spectrogram CNN model parameters
IMAGE_HEIGHT = img_size[0]
IMAGE_WIDTH = img_size[1]
N_CHANNELS = 3
N_CLASSES = 2

# #preprocess
# #     preprocess_images = np.log(images + 1e-10)
# preprocess_images = preprocess_resnet50(images)

def LOSO_cross_validation():
    for i in range(0, len(excerpts_data_folder_name)): 
        

        #leave on subject out as test data
        img_infront_test_subject = images[:subjects_data_index_mark[i,0],:,:,:]
        print(img_infront_test_subject.shape)
        label_infront_test_subject = labels[:subjects_data_index_mark[i,0]]
        img_behind_test_subject = images[subjects_data_index_mark[i,1]:,:,:,:]
        print(img_behind_test_subject.shape)
        label_behind_test_subject = labels[subjects_data_index_mark[i,1]:]

    #     if(label_infront_test_subject.shape[0]==0):
    #         training_img = img_behind_test_subject
    #         training_labels = label_behind_test_subject
    #     elif(img_behind_test_subject.shape[0]==0):
    #         training_img = label_infront_test_subject
    #         training_labels = label_infront_test_subject
    #     else:
    #         training_img = np.concatenate([img_infront_test_subject, img_behind_test_subject], axis=0)
    #         training_labels = np.concatenate([label_infront_test_subject, label_behind_test_subject], axis=0)
        training_img = np.concatenate([img_infront_test_subject, img_behind_test_subject], axis=0)
        training_labels = np.concatenate([label_infront_test_subject, label_behind_test_subject], axis=0)
        test_img = images[subjects_data_index_mark[i,0]:subjects_data_index_mark[i,1],:,:,:]
        test_labels = labels[subjects_data_index_mark[i,0]:subjects_data_index_mark[i,1]]
        
        # z score transform
        mean_value = training_img.mean()
        std_value = training_img.std()
        print(mean_value)
        print(std_value)
        print(training_img.shape)
        for j in range(training_img.shape[0]):
            training_img[j] = (training_img[j]-mean_value) / std_value

        print(test_img.shape)
        for j in range(test_img.shape[0]):
            test_img[j] = (test_img[j]-mean_value) / std_value

        #shuffle
        training_images,training_labels= shuffle(training_img, training_labels, random_state=0)
        test_images,test_labels= shuffle(test_img, test_labels, random_state=0)

        #validation data partition?

        #統計兩類數量
#         print(training_images.shape)
#     #     print(valid_images.shape)
#         print(test_images.shape)
#         print(training_labels.shape)
#     #     print(valid_labels.shape)
#         print(test_labels.shape)
#         train_count = 0
#     #     valid_count = 0
#         test_count = 0
#         for i in range(len(training_labels)):
#           if(training_labels[i]==1):
#             train_count += 1
#     #     for i in range(len(valid_labels)):
#     #       if(valid_labels[i]==1):
#     #         valid_count += 1
#         for i in range(len(test_labels)):
#           if(test_labels[i]==1):
#             test_count += 1

#         print(train_count)
#     #     print(valid_count)
#         print(test_count)


        #Construct model
        model = tf.keras.models.Sequential()
        model.add(tf.keras.layers.Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, N_CHANNELS)))
        model.add(tf.keras.layers.Conv2D(4, 3, strides=2, padding='same', activation='relu'))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Conv2D(8, 3, padding='same', activation='relu'))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Flatten())
        model.add(tf.keras.layers.Dense(16, activation='relu'))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Dropout(0.5))
        model.add(tf.keras.layers.Dense(N_CLASSES, activation='softmax'))
        # model.add(tf.keras.layers.Dense(N_CLASSES, activation='softmax', kernel_regularizer=regularizers.l2(0.01)))

        # Compile model
        model.compile(
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
            optimizer='adam',
    #         optimizer='SGD',
        #     optimizer=tf.keras.optimizers.legacy.RMSprop(),
            metrics=['accuracy'],
        )

        #star training
        history = model.fit(
            training_images,
            training_labels,
            epochs=200,
            batch_size=64,
    #         callbacks=[lr_sched],
            validation_data=(test_images, test_labels)
        )
    

In [ ]:
# Calculating the time consuming
import timeit

print(timeit.timeit(LOSO_cross_validation, number=1))

In [ ]:
#LOSO validation
# from tensorflow.keras.applications.vgg19 import preprocess_input as preprocess_vgg19
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_resnet50
from sklearn.utils import shuffle

# Define per-subject-left-out score containers
acc_per_subject_left_out = []
loss_per_subject_left_out = []


# Spectrogram CNN model parameters
IMAGE_HEIGHT = img_size[0]
IMAGE_WIDTH = img_size[1]
N_CHANNELS = 3
N_CLASSES = 2

from keras.callbacks import LearningRateScheduler
from keras import regularizers
###########################################
# DEFINE A STEPPED LEARNING RATE SCHEDULE #
###########################################
lr_sched = LearningRateScheduler(lambda epoch: 1e-4 * (0.75 ** np.floor(epoch / 2)))


#data partition
for i in range(0, len(excerpts_data_folder_name)): 
    #preprocess
#     preprocess_images = np.log(images + 1e-10)
    preprocess_images = preprocess_resnet50(images)
    
    #leave on subject out as test data
    img_infront_test_subject = preprocess_images[:subjects_data_index_mark[i,0],:,:,:]
    print(img_infront_test_subject.shape)
    label_infront_test_subject = labels[:subjects_data_index_mark[i,0]]
    img_behind_test_subject = preprocess_images[subjects_data_index_mark[i,1]:,:,:,:]
    print(img_behind_test_subject.shape)
    label_behind_test_subject = labels[subjects_data_index_mark[i,1]:]
    
#     if(label_infront_test_subject.shape[0]==0):
#         training_img = img_behind_test_subject
#         training_labels = label_behind_test_subject
#     elif(img_behind_test_subject.shape[0]==0):
#         training_img = label_infront_test_subject
#         training_labels = label_infront_test_subject
#     else:
#         training_img = np.concatenate([img_infront_test_subject, img_behind_test_subject], axis=0)
#         training_labels = np.concatenate([label_infront_test_subject, label_behind_test_subject], axis=0)
    training_img = np.concatenate([img_infront_test_subject, img_behind_test_subject], axis=0)
    training_labels = np.concatenate([label_infront_test_subject, label_behind_test_subject], axis=0)
    test_img = preprocess_images[subjects_data_index_mark[i,0]:subjects_data_index_mark[i,1],:,:,:]
    test_labels = labels[subjects_data_index_mark[i,0]:subjects_data_index_mark[i,1]]
    
    
    #shuffle
    training_images,training_labels= shuffle(training_img, training_labels, random_state=0)
    test_images,test_labels= shuffle(test_img, test_labels, random_state=0)
    
    #validation data partition?
    
    #統計兩類數量
    print(training_images.shape)
#     print(valid_images.shape)
    print(test_images.shape)
    print(training_labels.shape)
#     print(valid_labels.shape)
    print(test_labels.shape)
    train_count = 0
#     valid_count = 0
    test_count = 0
    for i in range(len(training_labels)):
      if(training_labels[i]==1):
        train_count += 1
#     for i in range(len(valid_labels)):
#       if(valid_labels[i]==1):
#         valid_count += 1
    for i in range(len(test_labels)):
      if(test_labels[i]==1):
        test_count += 1

    print(train_count)
#     print(valid_count)
    print(test_count)
    
    
    #Construct model
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, N_CHANNELS)))
    model.add(tf.keras.layers.Conv2D(4, 3, strides=2, padding='same', activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(8, 3, padding='same', activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(16, activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(N_CLASSES, activation='softmax'))
    # model.add(tf.keras.layers.Dense(N_CLASSES, activation='softmax', kernel_regularizer=regularizers.l2(0.01)))

    # Compile model
    model.compile(
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        optimizer='adam',
#         optimizer='SGD',
    #     optimizer=tf.keras.optimizers.legacy.RMSprop(),
        metrics=['accuracy'],
    )
    
    #star training
    history = model.fit(
        training_images,
        training_labels,
        epochs=200,
        batch_size=128,
#         callbacks=[lr_sched],
        validation_data=(test_images, test_labels)
    )
    
    
    # Generate generalization metrics
    scores = model.evaluate(test_images, test_labels, verbose=0)
    print(f'Score for subject{i+1} left out: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_subject_left_out.append(scores[1] * 100)
    loss_per_subject_left_out.append(scores[0])
    
    
    #Plot training, validation and test set accuracy
    print("Training Curve")
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()
    #Plot training, validation and test set loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()
    
    
    
    
    #  Generate confudion matrix(main function)
    y_predict = model.predict(test_images, batch_size=None, verbose=0, steps=None)
#     print(y_predict)
    y_pred = convert_to_labels(y_predict)
#     print(y_pred)
    y_true = test_labels
    target_names = ['No_spindle','With_spindle']
    print(classification_report(y_true, y_pred, target_names=target_names))
    print ("**************************************************************")
    plt.figure()
    cnf_matrix = confusion_matrix(y_true, y_pred)
    plot_confusion_matrix(cnf_matrix, classes=target_names, normalize=False, title='confusion matrix')
    plt.show()
    
    
    



# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per subject')
for i in range(0, len(acc_per_subject_left_out)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_subject_left_out[i]} - Accuracy: {acc_per_subject_left_out[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all subjects:')
print(f'> Accuracy: {np.mean(acc_per_subject_left_out)} (+- {np.std(acc_per_subject_left_out)})')
print(f'> Loss: {np.mean(loss_per_subject_left_out)}')
print('------------------------------------------------------------------------')
    
    
    
    
    
    
    
    

# Try with resnet50 to compare with our modedl

In [ ]:
# from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_resnet50
preprocess_images = preprocess_resnet50(images)

In [ ]:
# preprocess (log scaling)

print(images)
log_scaled_image = np.log(images + 1e-10)
print(log_scaled_image)

In [ ]:
# import cv2
# from google.colab.patches import cv2_imshow
# import os
# excerpt_data_path = r'/content/drive/MyDrive/DREAMS_data/SMOTE_5s_based_regenerated_balanced_data/compare_1D&2D/SMOTE_5s_epoch_based_with_without_REMs_phase_shift_80%_overlap_plot'
# excerpts_data_folder_name = os.listdir(excerpt_data_path)
# print(excerpts_data_folder_name)
# from natsort import natsorted
# print(natsorted(excerpts_data_folder_name))
# print(excerpts_data_folder_name)
# import os
# excerpt_label_path = r'/content/drive/MyDrive/DREAMS_data/SMOTE_5s_based_regenerated_balanced_data/compare_1D&2D/SMOTE_5s_epoch_based_with_without_REMs_80%_overlap_label'
# excerpt_label_txt_name = os.listdir(excerpt_label_path)
# print(excerpt_label_txt_name)
# from natsort import natsorted
# print(natsorted(excerpt_label_txt_name))
# print(excerpt_label_txt_name)

# Preprocess

In [ ]:
# shuffle DREAMS data

from sklearn.utils import shuffle
print(labels[:30])
preprocess_images,labels = shuffle(preprocess_images, labels, random_state=0)
# randomize = np.arange(len(labels_int))
# np.random.shuffle(randomize)
# x = x[randomize]
# y = y[randomize]
print(labels[:30])

In [ ]:
# shuffle DREAMS data

from sklearn.utils import shuffle
print(labels[:30])
images,labels = shuffle(images, labels, random_state=0)
# randomize = np.arange(len(labels_int))
# np.random.shuffle(randomize)
# x = x[randomize]
# y = y[randomize]
print(labels[:30])

In [ ]:
# DREAMS data(stage1,2)/label(stage1) whole partition(train:valid:test = 0.8:0.1:0.1)
train_amount = int(log_scaled_image.shape[0] * 80/100)
valid_amount = int(log_scaled_image.shape[0] * 10/100)
test_amount = int(log_scaled_image.shape[0] * 10/100)
train_images = log_scaled_image[0:train_amount,:,:,:]
train_labels = labels[0:train_amount]
valid_images = log_scaled_image[train_amount:train_amount + valid_amount,:,:,:]
valid_labels = labels[train_amount:train_amount + valid_amount]
test_images = log_scaled_image[train_amount + valid_amount:train_amount + valid_amount + test_amount,:,:,:]
test_labels = labels[train_amount + valid_amount:train_amount + valid_amount + test_amount]

In [ ]:
# DREAMS data(stage1,2)/label(stage1) whole partition(train:valid:test = 0.8:0.1:0.1)
train_amount = int(preprocess_images.shape[0] * 80/100)
valid_amount = int(preprocess_images.shape[0] * 10/100)
test_amount = int(preprocess_images.shape[0] * 10/100)
train_images = preprocess_images[0:train_amount,:,:,:]
train_labels = labels[0:train_amount]
valid_images = preprocess_images[train_amount:train_amount + valid_amount,:,:,:]
valid_labels = labels[train_amount:train_amount + valid_amount]
test_images = preprocess_images[train_amount + valid_amount:train_amount + valid_amount + test_amount,:,:,:]
test_labels = labels[train_amount + valid_amount:train_amount + valid_amount + test_amount]

In [ ]:
# DREAMS data(stage1,2)/label(stage1) whole partition(train:valid:test = 0.8:0.1:0.1)
train_amount = int(images.shape[0] * 80/100)
valid_amount = int(images.shape[0] * 10/100)
test_amount = int(images.shape[0] * 10/100)
train_images = images[0:train_amount,:,:,:]
train_labels = labels[0:train_amount]
valid_images = images[train_amount:train_amount + valid_amount,:,:,:]
valid_labels = labels[train_amount:train_amount + valid_amount]
test_images = images[train_amount + valid_amount:train_amount + valid_amount + test_amount,:,:,:]
test_labels = labels[train_amount + valid_amount:train_amount + valid_amount + test_amount]

In [ ]:
# z score normalization
train_images = train_images.astype('float64')
valid_images = valid_images.astype('float64')
test_images = test_images.astype('float64')
print(train_images.max())
print(train_images.min())

print(valid_images.max())
print(valid_images.min())

print(test_images.max())
print(test_images.min())

print(train_images.shape)
mean_value = train_images.mean()
std_value = train_images.std()
print(mean_value)
print(std_value)
for i in range(train_images.shape[0]):
    train_images[i,:,:,:] = (train_images[i,:,:,:]-mean_value) / std_value
            
# mean_value = valid_data.mean()
# std_value = valid_data.std()
# print(mean_value)
# print(std_value)
print(valid_images.shape)
for i in range(valid_images.shape[0]):
    valid_images[i,:,:,:] = (valid_images[i,:,:,:]-mean_value) / std_value
            
# mean_value = test_data.mean()
# std_value = test_data.std()
# print(mean_value)
# print(std_value)  
print(test_images.shape)
for i in range(test_images.shape[0]):
    test_images[i,:,:,:] = (test_images[i,:,:,:]-mean_value) / std_value
            
print(train_images.max())
print(train_images.min())

print(valid_images.max())
print(valid_images.min())

print(test_images.max())
print(test_images.min())

In [ ]:
print(train_images)

In [ ]:
print(train_images.shape)
print(valid_images.shape)
print(train_labels.shape)
print(valid_labels.shape)
train_count = 0
valid_count = 0
test_count = 0
for i in range(len(train_labels)):
  if(train_labels[i]==1):
    train_count += 1
for i in range(len(valid_labels)):
  if(valid_labels[i]==1):
    valid_count += 1
for i in range(len(valid_labels)):
  if(test_labels[i]==1):
    test_count += 1

print(train_count)
print(valid_count)
print(test_count)

# Fine-tuning model for training

In [ ]:
kernel_size2 = (5, 5)
kernel_size = (3, 3)
model = Sequential()
model.call = tf.function(model.call)
# The first two layers with 32 filters of window size 3x3
model.add(Conv2D(32, kernel_size, padding='same', activation='relu', input_shape=(256, 256, 3)))
model.add(Conv2D(32, kernel_size, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size, padding='same', activation='relu'))
model.add(Conv2D(64, kernel_size, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size, padding='same', activation='relu'))
model.add(Conv2D(64, kernel_size, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',classes=np.unique(train_labels),y=train_labels)

history = model.fit(train_images, train_labels, epochs=20, class_weight=class_weight,
                    validation_data=(valid_images, valid_labels))

In [ ]:
# ResNet50
from tensorflow.keras.applications import ResNet50

num_classes = 2    
pred_model = ResNet50(include_top=False, weights='imagenet', input_shape=(256, 256, 3), pooling='max', classifier_activation='softmax')
output_layer = Dense(num_classes, activation="softmax", name="output_layer")
model = tf.keras.Model(pred_model.inputs, output_layer(pred_model.output))
model.compile(optimizer=tf.keras.optimizers.Adam(),loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',classes=np.unique(train_labels),y=train_labels)

history = model.fit(train_images, train_labels, epochs=20, class_weight=class_weight,
                    validation_data=(valid_images, valid_labels))

In [ ]:
!pip install tensorflow==2.7.0
!pip install tf-models-official==2.7.0
!pip install tensorflow_io==0.23.1

# Try with VGG19 for compare

In [ ]:
from tensorflow.keras.applications import VGG19
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers
from tensorflow.keras import models
# load the pretrained model
base_model = tf.keras.applications.VGG19(
        input_shape=(256, 256, 3),
        include_top=False,
        weights="imagenet",
)

# freeze all layers of the model
base_model.trainable = False

# # we start by unfreezing all layers of the base model
# base_model.trainable = True

# # Freeze all layers except the 10 last layers 
# for layer in base_model.layers[:-10]: 
#     layer.trainable = False

# learning_rate = 1e-4

# # compile and retrain with a low learning rate
# # low_lr = learning_rate / 10


model = tf.keras.Sequential([
    tf.keras.Input(shape=(256, 256, 3)),
    # layers.Rescaling(scale=1.0/255), # rescale the pixel values to the [0, 1] range
    base_model,
    # add a new FC classifier on top of the base model
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(1, activation='sigmoid'),
])
learning_rate = 1e-4
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=learning_rate),
              metrics=['accuracy']
             )

from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',classes=np.unique(train_labels),y=train_labels)

epochs = 20
history = model.fit(train_images, train_labels, epochs=epochs, class_weight=class_weight,
                   validation_data=(valid_images, valid_labels))

In [ ]:
print(train_images.max())
print(train_images.min())

# Try with our target model

In [ ]:
# Spectrogram CNN model
IMAGE_HEIGHT = img_size[0]
IMAGE_WIDTH = img_size[1]
N_CHANNELS = 3
N_CLASSES = 2

from keras.callbacks import LearningRateScheduler
from keras import regularizers
###########################################
# DEFINE A STEPPED LEARNING RATE SCHEDULE #
###########################################

lr_sched = LearningRateScheduler(lambda epoch: 1e-4 * (0.75 ** np.floor(epoch / 2)))

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, N_CHANNELS)))
model.add(tf.keras.layers.Conv2D(4, 3, strides=2, padding='same', activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(8, 3, padding='same', activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Flatten())
# model.add(tf.keras.layers.Dense(4, activation='relu'))
# model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(N_CLASSES, activation='softmax'))
# model.add(tf.keras.layers.Dense(N_CLASSES, activation='softmax', kernel_regularizer=regularizers.l2(0.01)))

# Compile model
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer='adam',
#     optimizer='SGD',
#     optimizer=tf.keras.optimizers.RMSprop(),
    metrics=['accuracy'],
)

from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',classes=np.unique(train_labels),y=train_labels)

# history = model.fit(train_images, train_labels, epochs=100, batch_size=32, callbacks=[lr_sched], class_weight=class_weight, validation_data=(valid_images, valid_labels))
history = model.fit(train_images, train_labels, epochs=100, batch_size=256, 
#                     callbacks=[lr_sched], 
                    validation_data=(valid_images, valid_labels))

In [ ]:
model.summary()

In [ ]:
# plt.plot(history.history['accuracy'], label='accuracy')
# plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')
# plt.ylim([0.5, 1])
# plt.legend(loc='lower right')

# test_loss, test_acc = model.evaluate(valid_images, valid_labels, verbose=2)
# print(train_images)

#Plot training, validation and test set accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
#Plot training, validation and test set loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
#10 fold cross validation
from sklearn.model_selection import KFold
import numpy as np


# Spectrogram CNN model parameter
IMAGE_HEIGHT = img_size[0]
IMAGE_WIDTH = img_size[1]
N_CHANNELS = 3
N_CLASSES = 2


# Define per-fold score containers
acc_per_fold = []
loss_per_fold = []

# Merge inputs and targets
inputs = np.concatenate((train_images, valid_images), axis=0)
targets = np.concatenate((train_labels, valid_labels), axis=0)


# loss_function = sparse_categorical_crossentropy
# no_epochs = 25
# optimizer = Adam()
# # verbosity = 1
num_folds = 10

# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=True)


epochs=200
batch_size=64


preprocess_images = preprocess_resnet50(images)
def ten_folds_cross_validation():
    fold_no = 1
    for train, test in kfold.split(inputs, targets):
        
      # Define the model architecture
      #Construct model
        model = tf.keras.models.Sequential()
        model.add(tf.keras.layers.Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, N_CHANNELS)))
        model.add(tf.keras.layers.Conv2D(4, 3, strides=2, padding='same', activation='relu'))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Conv2D(8, 3, padding='same', activation='relu'))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Flatten())
        model.add(tf.keras.layers.Dense(16, activation='relu'))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Dropout(0.5))
        model.add(tf.keras.layers.Dense(N_CLASSES, activation='softmax'))
        # model.add(tf.keras.layers.Dense(N_CLASSES, activation='softmax', kernel_regularizer=regularizers.l2(0.01)))

        # Compile model
        model.compile(
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
            optimizer='adam',
    #         optimizer='SGD',
        #     optimizer=tf.keras.optimizers.legacy.RMSprop(),
            metrics=['accuracy'],
        )

        # Generate a print
        print('------------------------------------------------------------------------')
        print(f'Training for fold {fold_no} ...')

        # Fit data to model
        history = model.fit(inputs[train], targets[train],
                      batch_size=batch_size,
                      epochs=epochs)

        # Increase fold number
        fold_no = fold_no + 1
    
    
    

In [ ]:
# Calculating the time consuming
import timeit

print(timeit.timeit(ten_folds_cross_validation, number=1))

In [ ]:
# 1 prediction
def one_prediction():
    y_predict = model.predict(test_images, batch_size=None, verbose=0, steps=None)

In [ ]:
# 1 prediction time
import timeit

print(timeit.timeit(one_prediction, number=1000))

In [ ]:
#10 fold cross validation
from sklearn.model_selection import KFold
import numpy as np


# Spectrogram CNN model parameter
IMAGE_HEIGHT = img_size[0]
IMAGE_WIDTH = img_size[1]
N_CHANNELS = 3
N_CLASSES = 2

from keras.callbacks import LearningRateScheduler
from keras import regularizers
###########################################
# DEFINE A STEPPED LEARNING RATE SCHEDULE #
###########################################

lr_sched = LearningRateScheduler(lambda epoch: 1e-4 * (0.75 ** np.floor(epoch / 2)))


# Define per-fold score containers
acc_per_fold = []
loss_per_fold = []

# Merge inputs and targets
inputs = np.concatenate((train_images, valid_images), axis=0)
targets = np.concatenate((train_labels, valid_labels), axis=0)


# loss_function = sparse_categorical_crossentropy
# no_epochs = 25
# optimizer = Adam()
# # verbosity = 1
num_folds = 10

# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=True)


epochs=200
batch_size=64

# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(inputs, targets):
    
    
  # Define the model architecture
  #Construct model
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, N_CHANNELS)))
    model.add(tf.keras.layers.Conv2D(4, 3, strides=2, padding='same', activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(8, 3, padding='same', activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(16, activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(N_CLASSES, activation='softmax'))
    # model.add(tf.keras.layers.Dense(N_CLASSES, activation='softmax', kernel_regularizer=regularizers.l2(0.01)))

    # Compile model
    model.compile(
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        optimizer='adam',
#         optimizer='SGD',
    #     optimizer=tf.keras.optimizers.legacy.RMSprop(),
        metrics=['accuracy'],
    )
    
    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    # Fit data to model
    history = model.fit(inputs[train], targets[train],
                  batch_size=batch_size,
                  epochs=epochs)
    
    
    
    #  Generate confudion matrix(main function)
    y_predict = model.predict(inputs[test], batch_size=None, verbose=0, steps=None)
#     print(y_predict)
    y_pred = convert_to_labels(y_predict)
#     print(y_pred)
    y_true = targets[test]
    target_names = ['No_spindle','With_spindle']
    print(classification_report(y_true, y_pred, target_names=target_names))
    print ("**************************************************************")
    plt.figure()
    cnf_matrix = confusion_matrix(y_true, y_pred)
    plot_confusion_matrix(cnf_matrix, classes=target_names, normalize=False, title='confusion matrix')
    plt.show()
    
    
    # Generate generalization metrics
    scores = model.evaluate(inputs[test], targets[test], verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    # Increase fold number
    fold_no = fold_no + 1

# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
    print('------------------------------------------------------------------------')
    print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

In [ ]:
# Show the performance of testing data
results = model.evaluate(test_images,test_labels)
print(model.metrics_names)
print(results)

preds = model.predict(test_images[1:2,:,:,:])
print(preds)

In [ ]:
model.summary()

In [ ]:
from sklearn.metrics import classification_report

import itertools
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import numpy as np

# Confusion matrix
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
# 透過convert_to_labels 將one-hot-encode的資料轉成label
def convert_to_labels(X):
  return np.argmax(X, axis=1)

In [ ]:
#  Generate confudion matrix(main function)
y_predict = model.predict(test_images, batch_size=None, verbose=0, steps=None)
# print(y_predict)
y_pred = convert_to_labels(y_predict)
# print(y_pred)
y_true = test_labels
target_names = ['No_REMs','With_REMs']
print(classification_report(y_true, y_pred, target_names=target_names))
print ("**************************************************************")








plt.figure()
cnf_matrix = confusion_matrix(y_true, y_pred)
plot_confusion_matrix(cnf_matrix, classes=target_names, normalize=True, title='confusion matrix')

plt.show()

In [ ]:
def get_img_array(img_path, size):
    # `img` is a PIL image of size 299x299
    img = keras.preprocessing.image.load_img(img_path, target_size=size)
    # `array` is a float32 Numpy array of shape (299, 299, 3)
    array = keras.preprocessing.image.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    # of size (1, 299, 299, 3)
    array = np.expand_dims(array, axis=0)
    return array

def make_gradcam_heatmap(img_array, img_label, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)

    if(pred_index==1):
        if(img_label==1):
            result = 'Groundtruth: REMs/ Prediction: REMs'
        elif(img_label):
            result = 'Groundtruth: NoREMs/ Prediction: REMs'
    elif(pred_index==0):
        if(img_label==1):
            result = 'Groundtruth: REMs/ Prediction: NoREMs'
        elif(img_label==0):
            result = 'Groundtruth: NoREMs/ Prediction: NoREMs'


    return heatmap.numpy(),result

In [ ]:
misclassify_count = 0
correct_record = np.zeros((300,), dtype=int)
for i in range(0,493):
  test_img_matrix = test_images[i:i+1,:,:,:]
  test_img = test_images[i,:,:,:]
  test_img_label = test_labels[i,]
  heatmap, result = make_gradcam_heatmap(test_img_matrix, test_img_label, model, 'conv2d_5')
  # if (result == 'Groundtruth: NoSpindle/ Prediction: Spindle') | (result == 'Groundtruth: Spindle/ Prediction: NoSpindle'):
  if (result == 'Groundtruth: REMs/ Prediction: REMs'):
#     misclassify_count += 1
    print(heatmap.shape)
    print(i)
    correct_record[i] = 1
    # Display heatmap
    plt.matshow(heatmap)
    plt.show()


    heatmap = cv2.resize(heatmap,(test_img_matrix.shape[1],test_img_matrix.shape[2]), interpolation=cv2.INTER_CUBIC)
    # heatmap = np.uint8(255 * heatmap)
    fig, axs = plt.subplots(1, 1, sharex=True, sharey=True)
    test_img = np.float32(test_img)
    print(test_img.dtype)
    im = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)
#     cv2.imshow("windows_name", im)
    # 以 0.6 透明度繪製原始影像
    print(result)
    print('')
    axs.imshow(im, alpha=0.6)   
    # 以 0.4 透明度繪製熱力圖
#     axs.imshow(heatmap, cmap='jet', alpha=0.4)
  if (result == 'Groundtruth: REMs/ Prediction: NoREMs'):
    misclassify_count += 1
    print(heatmap.shape)
    print(i)
    correct_record[i] = 1
    # Display heatmap
    plt.matshow(heatmap)
    plt.show()


    heatmap = cv2.resize(heatmap,(test_img_matrix.shape[1],test_img_matrix.shape[2]), interpolation=cv2.INTER_CUBIC)
    # heatmap = np.uint8(255 * heatmap)
    fig, axs = plt.subplots(1, 1, sharex=True, sharey=True)
    test_img = np.float32(test_img)
    print(test_img.dtype)
    im = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)
#     cv2.imshow("windows_name", im)
    # 以 0.6 透明度繪製原始影像
    print(result)
    print('')
    axs.imshow(im, alpha=0.6)   
    # 以 0.4 透明度繪製熱力圖
#     axs.imshow(heatmap, cmap='jet', alpha=0.4)
print(misclassify_count)
for index, value in enumerate(correct_record):
  if value == 1:
    print(index)

In [ ]:
# Load DREAMS original data(no augment & shuffle)
import cv2
# from google.colab.patches import cv2_imshow
import os
# from natsort import natsorted
excerpt_data_path = r'/kaggle/input/5-sec-based-rems-regenerated-2d-data/5s_epoch_based_with_without_REMs_phase_shift_plot'
excerpts_data_folder_name = os.listdir(excerpt_data_path)
excerpts_data_folder_name = sorted(excerpts_data_folder_name)

positive_class_index = np.zeros((3240,), dtype=int)
img_count = 0
# load data
img_size = (128, 128)

img_count = 0

img_names = os.listdir(excerpt_data_path + '/' + excerpts_data_folder_name[0])
# img_names = sorted(img_names)
# print(img_names[0])

if img_names[0][-5] == '1':
    labels = 1
#     positive_class_index[img_count] = 
elif img_names[0][-5] == '0':
    labels = 0
else:
    print(img_names[0][-5])
img_count+=1
labels = np.expand_dims(labels, axis=0)  
print(labels.shape)
imgs = cv2.imread(excerpt_data_path + '/' + excerpts_data_folder_name[0] + '/' + img_names[0])
# cv2_imshow(images) 
imgs = cv2.resize(imgs, img_size, interpolation=cv2.INTER_AREA)
# cv2_imshow(images)
# print(images)
imgs = np.expand_dims(imgs, axis=0)
print(images.shape)
print(labels.shape)
# print(preprocess_images.shape)
for name in img_names[1:]:
#     print(name)
    if name[-5] == '1':
        label = 1
    elif name[-5] == '0':
        label = 0
    else:
        print(name[-5])
    label = np.expand_dims(label, axis=0)  
    print(labels.shape)
    print(label.shape)
    labels = np.append(labels, label, axis = 0)
    img = cv2.imread(excerpt_data_path + '/' + excerpts_data_folder_name[0] + '/' + name)
    img = cv2.resize(img, img_size, interpolation=cv2.INTER_AREA)
    img = np.expand_dims(img, axis=0)
    imgs = np.append(imgs, img, axis=0)
    print(imgs.shape)
    print(labels.shape)
    

for folder_name in excerpts_data_folder_name[1:]:
    img_names = os.listdir(excerpt_data_path + '/' + folder_name)
#     img_names = sorted(img_names)
    for name in img_names[0:]:
        print(name)
        if name[-5] == '1':
            label = 1
        elif name[-5] == '0':
            label = 0
        else:
            print(name[-5])
        label = np.expand_dims(label, axis=0)
        labels = np.append(labels, label, axis = 0)
        img = cv2.imread(excerpt_data_path + '/' + folder_name + '/' + name)
        img = cv2.resize(img, img_size, interpolation=cv2.INTER_AREA)
        img = np.expand_dims(img, axis=0)
        imgs = np.append(imgs, img, axis=0)
        print(imgs.shape)
        print(labels.shape)

# original_preprocess_images = preprocess_resnet50(images)

original_preprocess_images = imgs.astype('float64')

# z score normalization
for i in range(original_preprocess_images.shape[0]):
    original_preprocess_images[i,:,:,:] = (original_preprocess_images[i,:,:,:]-mean_value) / std_value

original_labels = labels
# print(preprocess_images)
# print(preprocess_images.shape)


In [ ]:
del img,imgs

In [ ]:
print(original_preprocess_images)

In [ ]:
print(original_preprocess_images.max())
print(original_preprocess_images.min())

In [ ]:
print(original_labels[2000:3000])

In [ ]:

for i in range(2000,3000):
    preds = model.predict(original_preprocess_images[i:i+1,:,:,:])
    pred_class = np.argmax(preds)
    print(pred_class)
    if((pred_class == original_labels[i]) and (original_labels[i]==1)):
        print("true positive sample")
        print(i)

In [ ]:
print(images)

In [ ]:
# 2d-cnn grad-cam 
from tensorflow.keras import models
import tensorflow.keras.backend as K
import cv2
import keras
import torch
import tensorflow.compat.v1 as tf

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm

last_conv_layer = model.get_layer(index=-7)
heatmap_model = models.Model([model.input], [last_conv_layer.output, model.output])

for i in range(1000,1100):
    input_test_img = original_preprocess_images[i:i+1,:,:,:]
    preds = model.predict(input_test_img)
    pred_class = np.argmax(preds)
    if((pred_class == original_labels[i]) and (original_labels[i]==1)):
        print("true positive sample")
        print(i)
        with tf.GradientTape() as gtape:
            conv_output, Predictions = heatmap_model(input_test_img)
            prob = Predictions[:, K.argmax(Predictions[0])]
            grads = gtape.gradient(prob, conv_output)
            pooled_grads = K.mean(grads, axis=(0,1,2)) 
  

        heatmap = tf.reduce_mean(tf.multiply(pooled_grads, conv_output), axis=-1)
        print(heatmap.shape)
        heatmap = K.maximum(heatmap,0)
        print(heatmap.shape)
        max_heat = K.max(heatmap)
        if max_heat == 0:
            max_heat = 1e-10
        heatmap /= max_heat
        
        heatmap_original = np.asarray(heatmap)
        
        plt.imshow(255*heatmap[0])
  
    
  
#         print(heatmap)
        im = cv2.resize(cv2.cvtColor(imgs[i,:,:,:], cv2.COLOR_BGR2RGB), (imgs[i,:,:,:].shape[1], imgs[i,:,:,:].shape[0]))
    
        # 拉伸 heatmap
        heatmap = cv2.resize(np.asarray(heatmap[0]), (imgs[i,:,:,:].shape[1], imgs[i,:,:,:].shape[0]))

        heatmap = np.uint8(255 * heatmap)


  
        

        # 分別plot         
        time = [0,1,2,3,4,5]
        default_x_ticks = [0,25.6,51.2,76.8,102.4,127]
        
        fig =plt.figure()
        ax1 = plt.subplot(1,3,1)
        ax1.set_xticks(default_x_ticks, time)
        ax1.set_yticklabels([])
        ax1.imshow(im, cmap='viridis')
        
        ax2 = plt.subplot(1,3,2)
        ax2.set_xticks(default_x_ticks, time)
        ax2.set_yticklabels([])
        ax2.imshow(heatmap, cmap='jet')
        
        ax3 = plt.subplot(1,3,3)
        ax3.set_xticks(default_x_ticks, time)
        ax3.set_yticklabels([])
        # 以 0.6 透明度繪製原始影像
        ax3.imshow(im, cmap='viridis', alpha=0.7)
        # 以 0.4 透明度繪製熱力圖
        ax3.imshow(heatmap, cmap='jet', alpha=0.3)
        
        if(pred_class==1):
            if(original_labels[i]==1):
                 fig.suptitle('Groundtruth: REM/ Prediction: REM')
            elif(original_labels[i]==0):
                 fig.suptitle('Groundtruth: NREM/ Prediction: REM')
        elif(pred_class==0):
            if(original_labels[i]==1):
                 fig.suptitle('Groundtruth: REM/ Prediction: NREM')
            elif(original_labels[i]==0):
                 fig.suptitle('Groundtruth: NREM/ Prediction: NREM')
#         plt.colorbar(ax2.add_collection(heatmap_original),ax=ax2)
        
        plt.show()


In [ ]:
 plt.figure()
plt.subplot(1,2,1)
plt.imshow(original_preprocess_images[i], cmap='viridis')
plt.subplot(1,2,2)
plt.imshow(heatmap, cmap='jet')
plt.show()

In [ ]:
def processing_image(img):
    # # 讀取影像為 PIL 影像
    # img = image.load_img(img_path, target_size=(224, 224))
    
    # # 轉換 PIL 影像為 nparray
    # x = image.img_to_array(img)
    
    # 加上一個 batch size，例如轉換 (224, 224, 3) 為 （1, 224, 224, 3) 
    img = np.expand_dims(img, axis=0)
    
    x = np.log(img + 1e-10)
#     x = preprocess_resnet50(img)
    
    return x

In [ ]:
def gradcam(model, x):
    # 取得影像的分類類別
    preds = model.predict(x)
    pred_class = np.argmax(preds)
    
    # # 取得影像分類名稱
    # pred_class_name = imagenet_utils.decode_predictions(preds)[0][0][1]
    
    # 預測分類的輸出向量
    pred_output = model.output[:, pred_class]
    
    # 最後一層 convolution layer 輸出的 feature map
    # ResNet 的最後一層 convolution layer
    last_conv_layer = model.get_layer(index=9)
    
    # 求得分類的神經元對於最後一層 convolution layer 的梯度
    grads = K.gradients(pred_output, last_conv_layer.output)[0]
    
    # 求得針對每個 feature map 的梯度加總
    pooled_grads = K.sum(grads, axis=(0, 1, 2))
    
    # K.function() 讓我們可以藉由輸入影像至 `model.input` 得到 `pooled_grads` 與
    # `last_conv_layer[0]` 的輸出值，像似在 Tensorflow 中定義計算圖後使用 feed_dict
    # 的方式。
    iterate = K.function([model.input], [pooled_grads, last_conv_layer.output[0]])
    
    # 傳入影像矩陣 x，並得到分類對 feature map 的梯度與最後一層 convolution layer 的 
    # feature map
    pooled_grads_value, conv_layer_output_value = iterate([x])
    
    # 將 feature map 乘以權重，等於該 feature map 中的某些區域對於該分類的重要性
    for i in range(pooled_grads_value.shape[0]):
        conv_layer_output_value[:, :, i] *= (pooled_grads_value[i])
        
    # 計算 feature map 的 channel-wise 加總
    heatmap = np.sum(conv_layer_output_value, axis=-1)
    
    return heatmap, pred_class

In [ ]:
def plot_heatmap(heatmap, img, pred_class, REM_or_NREM, fig_name):
    # ReLU
    heatmap = np.maximum(heatmap, 0)
    
    # 正規化
    heatmap /= np.max(heatmap)
    
    # # 讀取影像
    # img = cv2.imread(img_path)
    
    fig, ax = plt.subplots()
    
    im = cv2.resize(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), (img.shape[1], img.shape[0]))

    # 拉伸 heatmap
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))

    heatmap = np.uint8(255 * heatmap)
    
    # 以 0.6 透明度繪製原始影像
    ax.imshow(im, alpha=0.6)
    
    # 以 0.4 透明度繪製熱力圖
    ax.imshow(heatmap, cmap='jet', alpha=0.4)
    
    
    plt.title(str(pred_class))
    
    # Save figure
    figure_name = r'/content/drive/MyDrive/DREAMS_data/SMOTE_5s_based_regenerated_balanced_data/CAM_SMOTE_with_without_REMs_phase_shift_80%_overlap/'+ REM_or_NREM + '/' + fig_name
    plt.axis('off')
    plt.savefig(figure_name)

    plt.show()

    

In [ ]:
# plot specific Grad-CAM result
chosen = 26
heatmap, pred_class_name = gradcam(model, test_images[chosen:chosen+1,:,:,:])
img = test_images[chosen,:,:,:]
heatmap = np.maximum(heatmap, 0)
# 正規化
heatmap /= np.max(heatmap)
fig, ax = plt.subplots()
im = cv2.resize(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), (img.shape[1], img.shape[0]))
# 拉伸 heatmap
heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
heatmap = np.uint8(255 * heatmap)



# 分別plot 
# 偽坐標軸(只顯示座標，不scaling)
@plt.FuncFormatter
def fake_log(x, pos):
    'The two args are the value and tick position'
    return r'$2^{%d}$' % (x)
# input image
ax.imshow(im, extent=[0, 5, 0, 6], aspect=5/6, cmap='viridis')
# heat map
# ax.imshow(heatmap, extent=[0, 5, 0, 6], aspect=5/6, cmap='jet')

# 偽坐標軸(只顯示座標，不scaling)
ax.yaxis.set_major_formatter(fake_log)

plt.ylabel('Frequency (Hz)')
plt.xlabel('Time (s)')
plt.show()




#疊合
# # 偽坐標軸(只顯示座標，不scaling)
# @plt.FuncFormatter
# def fake_log(x, pos):
#     'The two args are the value and tick position'
#     return r'$2^{%d}$' % (x)

# # 以 0.6 透明度繪製原始影像
# ax.imshow(im, extent=[0, 5, 0, 6], aspect=5/6, alpha=0.6)
# # 以 0.4 透明度繪製熱力圖
# ax.imshow(heatmap, extent=[0, 5, 0, 6], aspect=5/6, cmap='jet', alpha=0.4)

# # 偽坐標軸(只顯示座標，不scaling)
# ax.yaxis.set_major_formatter(fake_log)

# plt.ylabel('Frequency (Hz)')
# plt.xlabel('Time (s)')
# plt.show()

In [ ]:
# show wrong classified plot
# pred = model.predict(preprocess_images)
# pred = np.argmax(pred, axis = 1)

for q in range(0, len(labels_int)):
  # 取得影像的分類類別
  img = processing_image(images[q,:,:,:])
  preds = model.predict(img)
  pred_class = np.argmax(preds)

  if((int(pred_class))!= (int(labels_int[q]))): 
    print(q)
    # # 取得影像分類名稱
    # pred_class_name = imagenet_utils.decode_predictions(preds)[0][0][1]
      
    # 預測分類的輸出向量
    pred_output = model.output[:, pred_class]
      
    # 最後一層 convolution layer 輸出的 feature map
    # ResNet 的最後一層 convolution layer
    last_conv_layer = model.get_layer(index=9)
      
    # 求得分類的神經元對於最後一層 convolution layer 的梯度
    grads = K.gradients(pred_output, last_conv_layer.output)[0]
      
    # 求得針對每個 feature map 的梯度加總
    pooled_grads = K.sum(grads, axis=(0, 1, 2))
      
    # K.function() 讓我們可以藉由輸入影像至 `model.input` 得到 `pooled_grads` 與
    # `last_conv_layer[0]` 的輸出值，像似在 Tensorflow 中定義計算圖後使用 feed_dict
    # 的方式。
    iterate = K.function([model.input], [pooled_grads, last_conv_layer.output[0]])
      
    # 傳入影像矩陣 x，並得到分類對 feature map 的梯度與最後一層 convolution layer 的 
    # feature map
    pooled_grads_value, conv_layer_output_value = iterate([img])
      
    # 將 feature map 乘以權重，等於該 feature map 中的某些區域對於該分類的重要性
    for i in range(pooled_grads_value.shape[0]):
        conv_layer_output_value[:, :, i] *= (pooled_grads_value[i])
          
    # 計算 feature map 的 channel-wise 加總
    heatmap = np.sum(conv_layer_output_value, axis=-1)



    # ReLU
    heatmap = np.maximum(heatmap, 0)
      
    # 正規化
    heatmap /= np.max(heatmap)

    print(int(labels_int[q]), " -> ", int(pred_class))
      
    # # 讀取影像
    # img = cv2.imread(img_path)
    
    im = cv2.resize(cv2.cvtColor(images[q,:,:,:], cv2.COLOR_BGR2RGB), (images[q,:,:,:].shape[1], images[q,:,:,:].shape[0]))
    
    # 拉伸 heatmap
    heatmap = cv2.resize(heatmap, (images[q,:,:,:].shape[1], images[q,:,:,:].shape[0]))

    heatmap = np.uint8(255 * heatmap)

    # 分別plot 
    plt.figure()
    plt.subplot(1,2,1)
    plt.imshow(im, cmap='viridis')
    plt.subplot(1,2,2)
    plt.imshow(heatmap, cmap='jet')
    plt.show()
    
    
    # # 疊在一起plot
    # plt.imshow(im, alpha=0.7)
    # plt.imshow(heatmap, cmap='jet', alpha=0.3)
    # plt.show()





In [ ]:
# Grad-CAM 主程式
for index,name in enumerate(REM_image_name):
  img = processing_image(REM_images[index,:,:])

  heatmap, pred_class_name = gradcam(model, img)

  plot_heatmap(heatmap, REM_images[index,:,:], pred_class_name, 'REMs', name)

for index,name in enumerate(NREM_image_name):
  img = processing_image(NREM_images[index,:,:])

  heatmap, pred_class_name = gradcam(model, img)

  plot_heatmap(heatmap, NREM_images[index,:,:], pred_class_name, 'NREMs', name)

In [ ]:
for i in range(200,220):
  # img = processing_image(images[i,:,:,:])

  heatmap, pred_class_name = gradcam(model, test_images[i:i+1,:,:,:])

  if(pred_class_name==1):
    if(test_labels[i]==1):
      plot_heatmap(heatmap, test_images[i,:,:,:], 'Prediction: REMs', 'REMs', str(i) + '(Groundtruth: REMs)')
    elif(test_labels[i]==0):
      plot_heatmap(heatmap, test_images[i,:,:,:], 'Prediction: REMs', 'REMs', str(i) + '(Groundtruth: NREMs)')
  elif(pred_class_name==0):
    if(test_labels[i]==1):
      plot_heatmap(heatmap, test_images[i,:,:,:], 'Prediction: NREMs', 'NREMs', str(i) + '(Groundtruth: REMs)')
    elif(test_labels[i]==0):
      plot_heatmap(heatmap, test_images[i,:,:,:], 'Prediction: NREMs', 'NREMs', str(i) + '(Groundtruth: NREMs)')

In [ ]:
print(test_labels)

In [ ]:
from sklearn.metrics import classification_report

import itertools
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import numpy as np

# Confusion matrix
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
# 透過convert_to_labels 將one-hot-encode的資料轉成label
def convert_to_labels(X):
  return np.argmax(X, axis=1)

In [ ]:
#  Generate confudion matrix(main function)
y_predict = model.predict(test_images, batch_size=None, verbose=0, steps=None)
print(y_predict)
y_pred = convert_to_labels(y_predict)
print(y_pred)
y_true = test_labels
target_names = ['No_REMs','With_REMs']
print(classification_report(y_true, y_pred, target_names=target_names))
print ("**************************************************************")

plt.figure()
cnf_matrix = confusion_matrix(y_true, y_pred)
plot_confusion_matrix(cnf_matrix, classes=target_names, normalize=True, title='confusion matrix')

plt.show()

In [ ]:
# Spectrogram CNN model (Stage2-with-REMs)
IMAGE_HEIGHT = img_size[0]
IMAGE_WIDTH = img_size[1]
N_CHANNELS = 3
N_CLASSES = 2

from keras.callbacks import LearningRateScheduler

###########################################
# DEFINE A STEPPED LEARNING RATE SCHEDULE #
###########################################

lr_sched = LearningRateScheduler(lambda epoch: 1e-4 * (0.75 ** np.floor(epoch / 2)))

model_REMs = tf.keras.models.Sequential()
model_REMs.add(tf.keras.layers.Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, N_CHANNELS)))
model_REMs.add(tf.keras.layers.Conv2D(32, 3, strides=2, padding='same', activation='relu'))
model_REMs.add(tf.keras.layers.BatchNormalization())
model_REMs.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model_REMs.add(tf.keras.layers.BatchNormalization())
model_REMs.add(tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'))
model_REMs.add(tf.keras.layers.BatchNormalization())
model_REMs.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model_REMs.add(tf.keras.layers.BatchNormalization())
model_REMs.add(tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'))
model_REMs.add(tf.keras.layers.BatchNormalization())
model_REMs.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model_REMs.add(tf.keras.layers.BatchNormalization())
model_REMs.add(tf.keras.layers.Flatten())
model_REMs.add(tf.keras.layers.Dense(256, activation='relu'))
model_REMs.add(tf.keras.layers.BatchNormalization())
model_REMs.add(tf.keras.layers.Dropout(0.5))
model_REMs.add(tf.keras.layers.Dense(N_CLASSES, activation='softmax'))

# Compile model
model_REMs.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.RMSprop(),
    metrics=['accuracy'],
)

history = model_REMs.fit(REMs_model_train_images, REMs_model_train_labels, epochs=15, batch_size=32, callbacks=[lr_sched], 
                    validation_data=(REMs_model_valid_images, REMs_model_valid_labels))

In [ ]:
# Spectrogram CNN model (Stage2-withour-REMs)
IMAGE_HEIGHT = img_size[0]
IMAGE_WIDTH = img_size[1]
N_CHANNELS = 3
N_CLASSES = 2

from keras.callbacks import LearningRateScheduler

###########################################
# DEFINE A STEPPED LEARNING RATE SCHEDULE #
###########################################

lr_sched = LearningRateScheduler(lambda epoch: 1e-4 * (0.75 ** np.floor(epoch / 2)))

model_no_REMs = tf.keras.models.Sequential()
model_no_REMs.add(tf.keras.layers.Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, N_CHANNELS)))
model_no_REMs.add(tf.keras.layers.Conv2D(32, 3, strides=2, padding='same', activation='relu'))
model_no_REMs.add(tf.keras.layers.BatchNormalization())
model_no_REMs.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model_no_REMs.add(tf.keras.layers.BatchNormalization())
model_no_REMs.add(tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'))
model_no_REMs.add(tf.keras.layers.BatchNormalization())
model_no_REMs.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model_no_REMs.add(tf.keras.layers.BatchNormalization())
model_no_REMs.add(tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'))
model_no_REMs.add(tf.keras.layers.BatchNormalization())
model_no_REMs.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model_no_REMs.add(tf.keras.layers.BatchNormalization())
model_no_REMs.add(tf.keras.layers.Flatten())
model_no_REMs.add(tf.keras.layers.Dense(256, activation='relu'))
model_no_REMs.add(tf.keras.layers.BatchNormalization())
model_no_REMs.add(tf.keras.layers.Dropout(0.5))
model_no_REMs.add(tf.keras.layers.Dense(N_CLASSES, activation='softmax'))

# Compile model
model_no_REMs.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.RMSprop(),
    metrics=['accuracy'],
)

history = model_no_REMs.fit(NREMs_model_train_images, NREMs_model_train_labels, epochs=15, batch_size=32, callbacks=[lr_sched], 
                    validation_data=(NREMs_model_valid_images, NREMs_model_valid_labels))

In [ ]:
stage2_1_input_images = np.zeros((1,256,256,3), dtype=np.double)
stage2_1_input_labels = np.zeros((1,), dtype=int)
stage2_2_input_images = np.zeros((1,256,256,3), dtype=np.double)
stage2_2_input_labels = np.zeros((1,), dtype=int)
count1 = 0
count0 = 0
for i in range(len(y_pred)):
  if(y_pred[i]==1):
    stage2_1_input_images = np.append(stage2_1_input_images, test_images[i:i+1,:,:,:],axis=0)
    stage2_1_input_labels = np.append(stage2_1_input_labels, test_labels[i:i+1],axis=0)
    count1 +=1
  elif(y_pred[i]==0):
    stage2_2_input_images = np.append(stage2_2_input_images, test_images[i:i+1,:,:,:],axis=0)
    stage2_2_input_labels = np.append(stage2_2_input_labels, test_labels[i:i+1],axis=0)
    count0 += 1

stage2_1_input_images = stage2_1_input_images[1:,:,:,:]
stage2_1_input_labels = stage2_1_input_labels[1:]
stage2_2_input_images = stage2_2_input_images[1:,:,:,:]
stage2_2_input_labels = stage2_2_input_labels[1:]



In [ ]:
from numpy.lib.function_base import append
#  Generate confudion matrix for stage2-with REMs classification(main function)
y_predict_REMs = model.predict(stage2_1_input_images, batch_size=None, verbose=0, steps=None)
y_predict_no_REMs = model.predict(stage2_2_input_images, batch_size=None, verbose=0, steps=None)
# print(y_predict_REMs)
# print(y_predict_no_REMs)
y_pred_REMs = convert_to_labels(y_predict_REMs)
y_pred_no_REMs = convert_to_labels(y_predict_no_REMs)
# print(y_pred_REMs)
# print(y_pred_no_REMs)
y_pred_final = np.append(y_pred_REMs,y_pred_no_REMs,axis=0)
# print(y_pred_final)
y_true_REMs = stage2_1_input_labels
y_true_no_REMs = stage2_2_input_labels
y_true_final = np.append(y_true_REMs, y_true_no_REMs,axis=0)
target_names = ['NREMs','REM']
print(classification_report(y_true_final, y_pred_final, target_names=target_names))
print ("**************************************************************")

plt.figure()
cnf_matrix = confusion_matrix(y_true_final, y_pred_final)
plot_confusion_matrix(cnf_matrix, classes=target_names, normalize=True, title='confusion matrix')

plt.show()

In [ ]:
# release the memory
del preprocess_images,images,train_images,train_labels,valid_images,valid_labels,test_images,test_labels
del model